<a href="https://colab.research.google.com/github/remre/TopicModelinLDAreddit/blob/main/redditclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deal with the Text in CoffeeHouse

##Libraries

In [8]:
!pip install pyLDAvis==3.2.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.2 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135617 sha256=df0d23ed3e00b378905a7218f2711242705700025cf459da58432a18ce97b36e
  Stored in directory: /root/.cache/pip/wheels/f8/b1/9b/560ac1931796b7303f7b517b949d2d31a4fbc512aad3b9f284
Successfully built pyLDAvis
  Attempting uninstall: pyLDAvis
    Found existing installation: pyLDAvis 3.3.1
    Uninstalling pyLDAvis-3.3.1:
      Successfully uninstalled pyLDAvis-3.3.1


In [1]:
# import os 
# # %matplotlib inline

# import time
# import glob

# import tensorflow as tf
# from tensorflow import keras
# import random
# # import stanza

# # from custom_preprocessing import CustomPreProcessing
# # from custom_preprocessing import PreProcessing
# # from class_metric import Metrics

# import sklearn
# from sklearn.utils import class_weight
# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn import decomposition, ensemble
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import accuracy_score
# from sklearn import tree
# from sklearn import neighbors
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import cohen_kappa_score
# from sklearn.metrics import precision_score
# from sklearn.model_selection import cross_validate
# from sklearn.linear_model import SGDClassifier

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# from keras.layers import Embedding
# from keras.layers import Conv1D, GlobalMaxPooling1D
# from keras.utils import np_utils


# import itertools
# from textblob import TextBlob 

# import xgboost as xgb
# from xgboost import XGBClassifier


# import string
# # import fasttext
# # import fasttext.util
# from tqdm import tqdm


# import pickle
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import RegexpTokenizer

# # DO = '#7D1B7E'

# # %config InlineBackend.figure_format = 'retina'
# # %matplotlib inline
# # plt.style.use('fivethirtyeight')

# # ---- Call tqdm to see progress bar with pandas
# # tqdm().pandas()
# import zipfile
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
# import pyLDAvis.gensim_models

import pandas as pd 
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
# os.environ["KAGGLE_CONFIG_DIR"] = ("/content")

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


##Data Loading##

We will load and check the data info in this section


In [2]:
!kaggle datasets download -d ehallmar/reddit-comment-score-prediction

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
# with zipfile.ZipFile('/content/reddit-comment-score-prediction.zip', 'r') as zip_ref:#the directory of zipped file
#     zip_ref.extractall('/content/drive/MyDrive/dataAI/redditdata') #directory of unzipped file

In [2]:
dk = pd.read_csv('/content/drive/MyDrive/dataAI/redditdata/comments_negative.csv')
df2 = pd.read_csv('/content/drive/MyDrive/dataAI/redditdata/comments_positive.csv')
dfr = pd.concat([dk,df2])

In [3]:
dfr.info()
dfr.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000000 entries, 0 to 1999999
Data columns (total 15 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   id                       object
 1   parent_id                object
 2   subreddit_id             object
 3   link_id                  object
 4   text                     object
 5   score                    int64 
 6   ups                      int64 
 7   author                   object
 8   controversiality         int64 
 9   parent_link_id           object
 10  parent_text              object
 11  parent_score             int64 
 12  parent_ups               int64 
 13  parent_author            object
 14  parent_controversiality  int64 
dtypes: int64(6), object(9)
memory usage: 488.3+ MB


id                          0
parent_id                   0
subreddit_id                0
link_id                     0
text                       53
score                       0
ups                         0
author                      0
controversiality            0
parent_link_id              0
parent_text                19
parent_score                0
parent_ups                  0
parent_author               0
parent_controversiality     0
dtype: int64

In [4]:
# 3
dfr['subreddit_id'].value_counts()

t5_2qh1i    706816
t5_2qh33    285724
t5_2qh0u    271402
t5_2qh61    169678
t5_2qh03    152591
             ...  
t5_2susi         1
t5_2s7u4         1
t5_2rufc         1
t5_30kkx         1
t5_2y0mz         1
Name: subreddit_id, Length: 5944, dtype: int64

In [5]:
dfr['author'] = dfr['author'].astype(str)

In [ ]:
# # 1
# type(json.loads(dfr['author']))

##Profanity Filter##

In [ ]:
from profanity_filter import ProfanityFilter
import spacy

pf = ProfanityFilter(languages=['en'])

pf.censor("That's bullshit!")
# "That's ********!"

k =  pf.censor_word('fuck')
# Word(uncensored='fuck', censored='****', original_profane_word='fuck')

In [ ]:
nlp = spacy.load('en')
profanity_filter = ProfanityFilter(nlps={'en': nlp})  # reuse spacy Language (optional)
nlp.add_pipe(profanity_filter.spacy_component, last=True)



for i in range(0,100):
  doc = nlp(str(dfr['text'][i]))

  doc._.is_profane
# True

  doc[:2]._.is_profane
# False

  for token in doc:
    if token._.original_profane_word != None:
        print(f'{token}: '
              f'\ncensored={token._.censored}, '
              f'\nis_profane={token._.is_profane}, '
              f'\noriginal_profane_word={token._.original_profane_word}'
        )

In [ ]:
# dfrr = dfr['text'][:3]
# train_token = ''
# for i in dfr['text']:
#    train_token += str(i)

In [ ]:
# this is how to add desired words by hard
# pf.extra_profane_word_dictionaries = {'en': {'chocolate', 'orange'}}

In [ ]:
from better_profanity import profanity


for i in range(0,100):
  doc = nlp(str(dfr['text'][i]))

  
  if __name__ == "__main__":
      # custom_badwords = ['happy', 'jolly', 'merry']
      # profanity.load_censor_words(custom_badwords)
      profanity.load_censor_words()

      for text in doc:
        censored_text = profanity.censor(text)# profanity.contains_profanity(str(text))
        print(censored_text)
        # You **** of ****.

In [ ]:
from profanity_filter import ProfanityFilter
# from rsrsrs.warmup.oopcorey import Employee
import spacy


class ProfanityChecker:

    def __init__(self, text):
        self.text = text 

    def proff(self):
        nlp = spacy.load('en')
        pf = ProfanityFilter(nlps={'en': nlp})  # reuse spacy Language (optional)
        nlp.add_pipe(pf.spacy_component, last=True)
        print(pf.spacy_component)
        # pf.set_censor_char('*')
        return pf.censor(self.text)



text1 = ProfanityChecker('what the fuck is this!!')
print(text1.proff())

##Data Processing##

##Text Classification and  Topic modeling

In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [11]:
# import bertopic
# from bertopic import BERTopic
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
dfr.head()

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c0a2d2p,t1_c0a2cn1,t5_1a8ah,t3_8pr4w,"Na, not really. \n\nI just hate islam and ever...",-2946,-2946,b34nz,0,t3_8pr4w,What goes through the heads of you people? Is...,459,459,[deleted],0
1,c6okok8,t1_c6oaywb,t5_2s8e9,t3_11otij,lol you're some ugly ass white dude,-2724,-2724,letmetellyouhowitis,0,t3_11otij,This is worth noting.\n\nThankfully I don't.,72,72,flowen65,0
2,c3nlalf,t1_c3nijr7,t5_2qzb6,t3_p9a1v,"First of off, its not true, and second off, I ...",-2132,-2132,iamwoodyharrelson,0,t3_p9a1v,I swear this is *(allegedly)* a true story. I...,4028,4028,AndyRooney,0
3,c10nh8q,t1_c10nc34,t5_6,t3_djasj,Who made you reddit police? I will submit what...,-2117,-2117,JimmyJamesincorp,0,t3_djasj,I was on my way over here to bitch and moan ab...,1214,1214,SloaneRanger,0
4,c3nlufk,t1_c3nlcob,t5_2qzb6,t3_p9a1v,We gotta be...i consider my time valuable.,-1962,-1962,iamwoodyharrelson,0,t3_p9a1v,Should change this AMA to AMAAR (Ask Me Anythi...,1405,1405,bersh,0


In [12]:
dfr['n_characters'] = dfr['text'].str.len()

In [13]:
dfr.query("n_characters>20").sort_values('n_characters', ascending=False)['text'][1]

1    Well, it is exactly what it sounds like. It's ...
1                  lol you're some ugly ass white dude
Name: text, dtype: object

In [13]:
dfr.loc[1]

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality,n_characters
1,c6okok8,t1_c6oaywb,t5_2s8e9,t3_11otij,lol you're some ugly ass white dude,-2724,-2724,letmetellyouhowitis,0,t3_11otij,This is worth noting.\n\nThankfully I don't.,72,72,flowen65,0,35.0
1,c4imcva,t1_c4im948,t5_2qh1i,t3_t0ynr,"Well, it is exactly what it sounds like. It's ...",9531,9531,Lynfect,0,t3_t0ynr,"Elaborate on this cum box, please.",3841,3841,eeeeevil,0,1746.0


In [ ]:
# df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
# df.head()

###Extract the text part from data set and process it
The processes include some of aplied text mining methods like lemmatzing, stemming, removing stopwords, n-grams etc

In [ ]:
# datam = df.content.values.tolist()

# # Remove Emails
# datam = [re.sub('\S*@\S*\s?', '', sent) for sent in datam]

# # Remove new line characters
# datam = [re.sub('\s+', ' ', sent) for sent in datam]

# # Remove distracting single quotes
# data_lemmatized = [re.sub("\'", "", sent) for sent in datam]

# print(datam[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15 I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL ---- brought to you by your neighborhood Lerxst ---- ']


In [14]:
data = dfr.text.sample(n = 50000).values.tolist()

In [17]:
for i in range(len(data)-2):
  if type(data[i]) != str:
    data.pop(i)
len(data)

50000

In [18]:
def remove_punctuation(text):
    return [tex.replace('[^\w\s]','') for tex in text]

nop = remove_punctuation(data)


In [ ]:
nop

In [19]:
def removesigns(text):
    for i in range(len(text)):
        text[i] = text[i].replace('.','').replace('/','').replace("'",'').replace('"','').replace(';','').replace(':','').replace('!','').replace(':','').replace('?','').replace('-','').replace('[','')

In [20]:
data = nop

In [69]:
print(nop[:25])

['uh, nice user name.\n\nNOT', 'Everything but the Brownie rinse is true. Rinsing it off is to much work. ', '/r/notinteresting', 'This is from an earlier question: "One time a lady asked me to throw a shrimp between her breasts. I kept refusing, but she kept insisting, so I obliged. Immediately her husband stood up and was like "What the hell?!?!?!" I was shocked for a second, then he busted out laughing and was like "I\'m just messing with you man." I then retired to the kitchen and changed my drawers."', "    Just because someone \n    is poor doesn't mean they have \n    to be messy though", "He'll take it in the end.", 'Apparently this is DOTA, not League. ', 'Finally, a use for all these mouse dicks. ', 'And the blonde chick with those gnarly green nails...cringeworthy. Those nails aint stepping anywhere near my controllers.', 'You wouldn\'t even know you I was saying if you had the filter on.\n    "I love big **** ***** " ... " Cause I\'m a stereotypical *** " ... " I\'m a *** i

###Tokenizer
it will seperate the text by tokens

In [21]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['forbidden', 'loaf']]


###N-grams and Lemmatizing

In [22]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['forbidden', 'loaf']


In [23]:
print(trigram_mod[bigram_mod[data_words[0]]])

['forbidden', 'loaf']


In [24]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result


In [25]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['forbid', 'loaf']]


In [ ]:
# #LET'S REPEAT STEPS THORUGH THERE FOR REDDIT TEXTS 
# # Remove Stop Words
# data_words_nostops = remove_stopwords(data_words)

# # Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# print(data_lemmatized[:1])

###Corpus and Dictionary

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)


### Remove very rare and very common words:

In [ ]:

# - words appearing less than 15 times
# - words appearing in more than 10% of all documents

# id2word.filter_extremes(no_below=5, no_above=0.1, keep_n= 100000)

In [27]:

# Create Corpus

texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
print(id2word[0])

[[(0, 1), (1, 1)]]
forbid


In [28]:
document_num = 2
bow_doc_x = corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     id2word[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 2 ("business") appears 2 time.
Word 3 ("constraint") appears 1 time.
Word 4 ("consumer") appears 1 time.
Word 5 ("cost") appears 1 time.
Word 6 ("dedicate") appears 1 time.
Word 7 ("delicious") appears 1 time.
Word 8 ("dish") appears 1 time.
Word 9 ("factory") appears 1 time.
Word 10 ("favorite") appears 1 time.
Word 11 ("feasibility") appears 1 time.
Word 12 ("food") appears 1 time.
Word 13 ("fortunately") appears 1 time.
Word 14 ("know") appears 1 time.
Word 15 ("large") appears 2 time.
Word 16 ("look") appears 1 time.
Word 17 ("make") appears 2 time.
Word 18 ("nutritional") appears 1 time.
Word 19 ("organize") appears 1 time.
Word 20 ("picture") appears 1 time.
Word 21 ("portion") appears 3 time.
Word 22 ("possible") appears 1 time.
Word 23 ("product") appears 1 time.
Word 24 ("refrain") appears 1 time.
Word 25 ("say") appears 1 time.
Word 26 ("service") appears 1 time.
Word 27 ("space") appears 1 time.
Word 28 ("spicy") appears 1 time.
Word 29 ("stay") appears 1 time.
Word 30 

In [ ]:
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [29]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
al_lda_model =  gensim.models.LdaMulticore(corpus, 
                                   num_topics = 8, 
                                   id2word = id2word,                                    
                                   passes = 10,
                                   workers = 2)

In [32]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in al_lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.008*"make" + 0.007*"good" + 0.007*"see" + 0.007*"time" + 0.005*"year" + 0.005*"actually" + 0.005*"even" + 0.005*"show" + 0.005*"work" + 0.005*"great"


Topic: 1 
Words: 0.018*"amp" + 0.008*"use" + 0.008*"baby" + 0.008*"sex" + 0.006*"eat" + 0.006*"food" + 0.006*"nice" + 0.005*"cry" + 0.005*"pic" + 0.005*"penis"


Topic: 2 
Words: 0.056*"com" + 0.020*"reddit" + 0.012*"http" + 0.010*"suite_http" + 0.010*"reddit_enhancement" + 0.008*"gif" + 0.008*"dog" + 0.007*"apple" + 0.007*"user" + 0.007*"link"


Topic: 3 
Words: 0.030*"get" + 0.021*"go" + 0.012*"time" + 0.008*"know" + 0.008*"say" + 0.008*"make" + 0.008*"day" + 0.007*"work" + 0.007*"year" + 0.007*"really"


Topic: 4 
Words: 0.024*"think" + 0.022*"people" + 0.016*"woman" + 0.012*"man" + 0.009*"say" + 0.009*"way" + 0.009*"make" + 0.008*"well" + 0.007*"really" + 0.007*"thing"


Topic: 5 
Words: 0.032*"game" + 0.020*"play" + 0.017*"make" + 0.012*"team" + 0.012*"see" + 0.010*"look" + 0.009*"player" + 0.008*"take" + 0.008*"

In [33]:
num = 55
unseen_document = data[num]
print(unseen_document)

I personally don't think so, but just to play devils advocate -- AP will be 30 years old by the start of next season at a position which is by far the quickest to age, and he'll be well over a year removed from any actual playing time, so really wouldn't be *too* crazy if it worked out like that. And that's not even counting the PR stuff that's unfortunately going to go along with it for whoever he lands with.


In [34]:
bow_vector = id2word.doc2bow(preprocess(unseen_document))

for index, score in sorted(al_lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, al_lda_model.print_topic(index, 5)))

Score: 0.5135196447372437	 Topic: 0.030*"get" + 0.021*"go" + 0.012*"time" + 0.008*"know" + 0.008*"say"
Score: 0.206903874874115	 Topic: 0.032*"game" + 0.020*"play" + 0.017*"make" + 0.012*"team" + 0.012*"see"
Score: 0.19013315439224243	 Topic: 0.024*"think" + 0.022*"people" + 0.016*"woman" + 0.012*"man" + 0.009*"say"
Score: 0.06309060752391815	 Topic: 0.018*"amp" + 0.008*"use" + 0.008*"baby" + 0.008*"sex" + 0.006*"eat"


In [35]:
# Compute Perplexity
print('\nPerplexity: ', al_lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_allda = CoherenceModel(model=al_lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_allda = coherence_model_allda.get_coherence()
print('\nCoherence Score: ', coherence_allda)


Perplexity:  -8.237456575684595

Coherence Score:  0.5483343282796052


In [36]:
pyLDAvis.enable_notebook()
visal = pyLDAvis.gensim.prepare(al_lda_model, corpus, id2word)
visal

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.117893 -0.011672       1        1  25.560716
7      0.105572  0.032451       2        1  16.042548
4      0.119872 -0.004051       3        1  14.358218
6      0.078375  0.051245       4        1  14.256761
0      0.063360  0.045454       5        1  11.871184
5      0.040255 -0.007585       6        1   8.967464
1     -0.186782 -0.289842       7        1   4.571831
2     -0.338546  0.184000       8        1   4.371279, topic_info=        Term         Freq        Total Category  logprob  loglift
217      com  1530.000000  1530.000000  Default  30.0000  30.0000
202     fuck  3077.000000  3077.000000  Default  29.0000  29.0000
463     game  1919.000000  1919.000000  Default  28.0000  28.0000
833   reddit  1792.000000  1792.000000  Default  27.0000  27.0000
184     play  1356.000000  1356.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
788   source    78.868570   278.000281   Topic8  -5.8462   1.8703
282     call    96.187448  1540.625334   Topic8  -5.6477   0.3565
656    video    86.587194   608.854252   Topic8  -5.7528   1.1797
507     edit    86.796139  1549.682929   Topic8  -5.7504   0.2479
1305    kill    82.604827   977.423211   Topic8  -5.7999   0.6593

[627 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1415      2  0.990615  abortion
506       1  0.278448   account
506       4  0.314483   account
506       5  0.032759   account
506       6  0.039310   account
...     ...       ...       ...
342       3  0.036592      year
342       5  0.261283      year
4038      6  0.281575   youtube
4038      8  0.708480   youtube
8041      3  0.977167    zombie

[1485 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 5, 7, 1, 6, 2, 3])

In [ ]:
# Let's Extend the number of topics and get more Remarkable ones now!

In [37]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [38]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.095*"need" + 0.036*"stupid" + 0.034*"thank" + 0.032*"support" + '
  '0.027*"free" + 0.023*"gun" + 0.021*"spend" + 0.019*"poor" + 0.019*"realize" '
  '+ 0.018*"other"'),
 (1,
  '0.083*"work" + 0.051*"life" + 0.038*"job" + 0.037*"pay" + 0.030*"end" + '
  '0.026*"already" + 0.019*"hour" + 0.018*"player" + 0.017*"business" + '
  '0.017*"walk"'),
 (2,
  '0.053*"play" + 0.042*"believe" + 0.032*"story" + 0.031*"state" + '
  '0.028*"die" + 0.021*"week" + 0.020*"far" + 0.018*"learn" + 0.017*"break" + '
  '0.016*"full"'),
 (3,
  '0.066*"well" + 0.063*"really" + 0.045*"give" + 0.044*"actually" + '
  '0.030*"downvote" + 0.021*"fact" + 0.021*"girl" + 0.020*"enough" + '
  '0.019*"less" + 0.018*"change"'),
 (4,
  '0.213*"say" + 0.086*"man" + 0.032*"real" + 0.027*"true" + 0.021*"answer" + '
  '0.014*"fun" + 0.013*"totally" + 0.013*"mom" + 0.010*"science" + '
  '0.010*"correct"'),
 (5,
  '0.044*"live" + 0.043*"read" + 0.024*"fucking" + 0.022*"human" + 0.021*"sex" '
  '+ 0.021*"animal" + 0.019

In [39]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.231317829827814

Coherence Score:  0.4022807987143701


In [40]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.351719 -0.262217       1        1  10.752091
11    -0.203242  0.369651       2        1   9.364605
8     -0.115340  0.047754       3        1   7.857336
3     -0.041574  0.015256       4        1   6.532763
17    -0.024243  0.016747       5        1   5.881435
15    -0.007529  0.001352       6        1   5.749781
14     0.023937 -0.010685       7        1   4.826078
6      0.033428 -0.010521       8        1   4.698258
12     0.037061 -0.009737       9        1   4.490411
16     0.033811 -0.011873      10        1   4.490027
1      0.039101 -0.012126      11        1   4.157075
2      0.048509 -0.012635      12        1   3.992348
13     0.057543 -0.013368      13        1   3.951345
4      0.052205 -0.011710      14        1   3.932619
9      0.060519 -0.016333      15        1   3.611447
5      0.066009 -0.017669      16        1   3.570209
0      0.064021 -0.013278      17        1   3.444839
10     0.070930 -0.014983      18        1   3.337436
7      0.075929 -0.016311      19        1   3.132764
19     0.080644 -0.017314      20        1   2.227132, topic_info=          Term         Freq        Total Category  logprob  loglift
130     people  6390.000000  6390.000000  Default  30.0000  30.0000
122        get  7854.000000  7854.000000  Default  29.0000  29.0000
25         say  5228.000000  5228.000000  Default  28.0000  28.0000
17        make  5302.000000  5302.000000  Default  27.0000  27.0000
303         go  5706.000000  5706.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2680     reach    94.071668    94.975849  Topic20  -4.9956   3.7949
747   property    93.295911    94.200092  Topic20  -5.0038   3.7948
4189      dota    89.629436    90.533616  Topic20  -5.0439   3.7944
2317  friendly    84.456521    85.360701  Topic20  -5.1034   3.7938
4190    esport    81.768021    82.672253  Topic20  -5.1357   3.7935

[630 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2821      1  0.997599     able
506      15  0.997668  account
671       9  0.997958      act
929       6  0.994645   action
252      12  0.996189   actual
...     ...       ...      ...
2170     13  0.996327    worry
862       2  0.996775    write
734       3  0.999550    wrong
342      10  0.999447     year
2601     11  0.996362    young

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 12, 9, 4, 18, 16, 15, 7, 13, 17, 2, 3, 14, 5, 10, 6, 1, 11, 8, 20])

In [1]:
import zipfile
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2022-08-17 19:47:15--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-08-17 19:47:16--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.1’

mallet-2.0.8.zip.1  100%[===================>]  15.43M  16.1MB/s    in 1.0s    

2022-08-17 19:47:17 (16.1 MB/s) - ‘mallet-2.0.8.zip.1’ saved [16184794/16184794]



In [2]:
with zipfile.ZipFile('/content/mallet-2.0.8.zip', 'r') as zip_ref:#the directory of zipped file
    zip_ref.extractall('/content/drive/MyDrive/dataAI/mallet') #directory of unzipped file


KeyboardInterrupt: ignored

In [3]:
import gensim

In [4]:
mallet_path = '/content/drive/MyDrive/dataAI/mallet/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

NameError: ignored

In [ ]:
import os

os.environ.update({
        'MALLET_HOME':
        r"/content/drive/MyDrive/dataAI/mallet/mallet-2.0.8/bin/mallet",
    })

In [ ]:
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")
all_stopwords = nlp.Defaults.stop_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."
lists = []
text_tokens = word_tokenize(data)
tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
def remove_stopwords(text):
  return [word for word in text_tokens if not word in all_stopwords]

remove_stopwords(text_tokens)

In [ ]:
tokens_without_sw

In [ ]:
# from _typeshed import Self
import requests, re
from bs4 import BeautifulSoup
from collections import Counter
import statistics as stats
import string
import pandas as pd
# %matplotlib inline
import numpy as np
import pandas as pd
import json 
import spacy
import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:


pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

# remove punc +
# remove stopwords +
# tokenize +
# lemmatizing - 
# stemmer - 

# 
# 
# 
# 


# create your class here
class ATextM:
   

    def __init__(self, src, src_type=None):
       
        self._src = src
        self._src_type = src_type
        self._orig_content = None
        self._content = None
        if self._src_type is None:
            print(
                'Source type not specified. Use discover() method to determine src_type.'
            )

            self._src_type = self.discover()
            self.fetch_by_src_type(self._src_type)
        self.fetch_by_src_type(self._src_type)

    
    def discover(self):
        
        if self._src.startswith('http'):
            _src_type = 'url'
        elif self._src.endswith('txt'):
            _src_type = 'path'
        elif self._src.endswith('json'):
             _src_type = 'json'
        else:
            _src_type = 'text'
        print(f'src_type from discover() method is {_src_type}.')
        return _src_type
        # self.fetch_by_src_type(self._src_type)
        

    def fetch_by_src_type(self, src_type):  # Do I really need to pass src_type into fetch?
       
        self._src_type = src_type
        # select correct read method for source type
        if self._src_type == 'url':
            headers = {'user-agent': 'my-app/0.0.1'}
            req = requests.get(self._src, headers=headers)
            self._orig_content = req.text
            self._content = self._orig_content
        elif self._src_type == 'path':
            with open(self._src, 'r') as f:
                self._content = f.read()

        elif self._src_type == 'json':
            with open(self._src, 'r') as f:
                
                self._content =  json.load(f)
                
                # Iterating through the json
                # list
                for i in self._content:
                    print(i)
                
                # Closing file
                f.close()
                
        elif self._src_type == 'text':
            self._content = self._src
        else:
            print('Supported source types: URL, text files and text strings.')
        
        
    @property
    def orig_content(self):
        """Property to directly access results from fetch_by_src_type
        method."""
        return self._orig_content

    
    def set_content_to_tag(self, tag='div', tag_id='content-main'):
        """Method uses beautiful soup library to parse HTML for a given
        tag and id after call to fetch_by_src_type(src_type='url). 
        Throws ValueError exception if tag not found. 
        Method is only used if src_type is URL.

        Keyword arguments:
        tag('str') -- HTML tag name
        tag_id('str') -- HTML tag id
        
        Grader: return a value but it is never used."""
        self._tag = tag
        self._tag_id = tag_id
        if self._tag != tag or self._tag_id != tag_id:
            print('Calling reset_content()')
            self.reset_content()
        else:
            try:
                # create beautiful soup object for tag and id
                soup = BeautifulSoup(self._content, 'html.parser')
                # get text and assign to _content attribute
                _content = soup.find(tag, {'id': tag_id}).get_text()
                self._content = _content
                # print(self._content)
            except ValueError:
                # raise exception of tag or tag_id is not found
                if tag is None or tag_id is None:
                    print(f"Tag {tag} or Tag_id {tag_id} is not found.")

    
    def reset_content(self):
        """Reset _content attribute to _orig_content fetched by requests
        module. Useful after call to set_content_to_tag method, e.g. if
        you want to use a different tag or id to analyze the same URL.
        
        Use when you want to create a new beautiful soup object based on
        a new tag or tag id using set_content_to_tag method. Saves call
        to requests module since html content is already in
        _orig_content attribute.
        
        Grader: return a value but it is never used."""
        if self._src_type == 'url':
            self._content = self._orig_content
            # confirm reset to _orig_content
            print('confirm reset_content: ', self._content[0])
            return self._content
        else:
            print(f'Source type {self._src_type} is not a URL.')
  
    def _words(self, casesensitive=False):
        """If casesensitive=False: returns list of words in upper case.
        If casesensitive=True: returns list of words from _orig_content.
        Returns list of words.
        
        Keyword arguments: 
        casesensitive (bool) -- Returns uppercase words if False
        (default).
        
        Grader: I've corrected your words method."""
        # print('_words(): self._content:\n', self._content)
        return (
            [
                word.strip(string.punctuation).lower()
                for word in self._content.split()
            ]
            if casesensitive is False
            else [word.strip(string.punctuation) for word in self._content.split()]
        )
    def common_words(self,words, minlen=1, maxlen=100, count=10, casesensitive=False):
        """Returns list of 2-element tuples in the format: (word: num)
        where num is a count of word occurences in the content obtained
        from the _words method using Counter().most_common() method
        from collections module.
        
        Keyword arguments:
        minlen (int) -- Minimum length for word to be included.
        maxlen (int) -- Maximum length for word to be included.
        count (int) -- Number of words to include in the results.
        casesensitive (bool) -- Returns uppercase words if False
        (default)."""
        self.words = words
        self.minlen = minlen
        print('self.minlen:', self.minlen)
        self.maxlen = maxlen
        print('self.maxlen:', self.maxlen)
        self.count = count
        print('self.count:', self.count)
        self.casesensitive = casesensitive
        print('self.casesensitive:', self.casesensitive)
        _word_list = []
        if casesensitive is True:
            _word_list = [w.lover() for w in self.words if (len(w) >= self.minlen) & (len(w) <= self.maxlen)]
            # print('self.words: casesensitive is False\n', _word_list)
        else:
            _word_list = [w for w in self.words if (len(w) >= self.minlen) & (len(w) <= self.maxlen)]
            # print('_word_list: casesensitive is True\n', _word_list)
        # use Counter from collections module
        cnt = Counter(_word_list)
        return cnt.most_common(count)
class WordPro:
    listt = []
    lists = []
    nlp = spacy.load('en_core_web_sm')
    all_stopwords = nlp.Defaults.stop_words
    text_tokens = word_tokenize(data)

    def __init__(self,text):
        self.text = text

    def remove_punctuation(self):
        return ' '.join(word.strip(string.punctuation) for word in self.text.split())
        # return self.text.replace('[^\w\s]','')

# Create a function to count the non punctuation token of a text

    def count_tokens(self):

        for i in range(len(self.text)):
            k  = self.nlp(str(self.text[i]))
            self.listt.append(len(k))
        return self.listt

    def tokenizer(self):
        text_tokens = word_tokenize(self.text)
        stemm = nltk.PorterStemmer()
        
        return [stemm.stem(t) for t in text_tokens]
                
        # pass
        
# topics_data['N_o_To'] = count_tokens(topics_data['title'])




# test and debug class
jjson = 'C:/Users/emreb/Documents/newoneforreco/rsrsrs/classificationt&tranasformsers/dbs/sample2.json'
url = 'https://senior.ceng.metu.edu.tr/2014/dcengounchained/SDD_DcengoUnchained_V1.1%20(3).pdf'
path = 'pride-and-prejudice.txt'


text = '''The outlook wasn't brilliant for the Mudville Nine that day;
the score stood four to two, with but one inning more to play.
And then when Cooney died at first, and Barrows did the same,
a sickly silence fell upon the patrons of the game.'''
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."

# t = ATextM(jjson, src_type='json')
ta = ATextM(url, src_type='url')
taa = ATextM(text, src_type='text')


# ww = taa._words()
# print(taa.common_words(ww))


kl = WordPro(data)
kl.tokenizer()

['In',
 'the',
 'town',
 'of',
 'athi',
 'one',
 'jeremi',
 'lanigan',
 'batter',
 'away',
 'til',
 'he',
 'hadnt',
 'a',
 'pound',
 '.',
 'hi',
 'father',
 'die',
 'and',
 'made',
 'him',
 'a',
 'man',
 'again',
 'left',
 'him',
 'a',
 'farm',
 'and',
 'ten',
 'acr',
 'of',
 'ground',
 '.',
 'He',
 'gave',
 'a',
 'grand',
 'parti',
 'for',
 'friend',
 'and',
 'relat',
 'who',
 'didnt',
 'forget',
 'him',
 'when',
 'come',
 'to',
 'the',
 'wall',
 ',',
 'and',
 'if',
 'youll',
 'but',
 'listen',
 'ill',
 'make',
 'your',
 'eye',
 'glisten',
 'Of',
 'the',
 'row',
 'and',
 'the',
 'ruction',
 'of',
 'lanigan',
 'ball',
 '.',
 'myself',
 'to',
 'be',
 'sure',
 'got',
 'free',
 'invit',
 ',',
 'for',
 'all',
 'the',
 'nice',
 'girl',
 'and',
 'boy',
 'I',
 'might',
 'ask',
 ',',
 'and',
 'just',
 'in',
 'a',
 'minut',
 'both',
 'friend',
 'and',
 'relat',
 'were',
 'danc',
 'round',
 'merri',
 'as',
 'bee',
 'round',
 'a',
 'cask',
 '.',
 'judi',
 'odali',
 ',',
 'that',
 'nice',
 'littl'

In [ ]:
kl.remove_punctuation()

'In the town of Athy one Jeremy Lanigan Battered away til he hadnt a pound His father died and made him a man again Left him a farm and ten acres of ground He gave a grand party for friends and relations Who didnt forget him when come to the wall And if youll but listen Ill make your eyes glisten Of the rows and the ructions of Lanigans Ball Myself to be sure got free invitation For all the nice girls and boys I might ask And just in a minute both friends and relations Were dancing round merry as bees round a cask Judy ODaly that nice little milliner She tipped me a wink for to give her a call And I soon arrived with Peggy McGilligan Just in time for Lanigans Ball There were lashings of punch and wine for the ladies Potatoes and cakes there was bacon and tea There were the Nolans Dolans OGradys Courting the girls and dancing away Songs they went round as plenty as water The harp that once sounded in Taras old hall Sweet Nelly Gray and The Rat Catchers Daughter All singing together at L